In [1]:
!sudo pip3 install --upgrade pip keras==2.1.5 tensorflow==1.13.1 numpy pandas pillow sklearn opencv-python optuna scikit-image  optkeras h5py==2.10.0

     |████████████████████████████████| 1.5 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 334 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 92.5 MB 385 kB/s  eta 0:00:01   |█                               | 3.0 MB 1.5 MB/s eta 0:00:59     |██████▍                         | 18.3 MB 2.6 MB/s eta 0:00:29     |████████▏                       | 23.5 MB 3.2 MB/s eta 0:00:22     |████████████████▋               | 48.1 MB 2.0 MB/s eta 0:00:23
     |████████████████████████████████| 14.8 MB 26.1 MB/s eta 0:00:01     |██████████████████████████████  | 13.9 MB 26.1 MB/s eta 0:00:01
     |████████████████████████████████| 9.5 MB 20.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 50.4 MB 33.1 MB/s eta 0:00:01   |███▎                            | 5.1 MB 8.7 MB/s eta 0:00:06     |████████████████████████████▌   | 44.9 MB 33.1 MB/s eta 0:00:01     |█████████████████████████████▏  | 45.9 M

     |████████████████████████████████| 1.1 MB 1.6 MB/s eta 0:00:01
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
Using legacy 'setup.py install' for termcolor, since package 'wheel' is not installed.
Using legacy 'setup.py install' for alembic, since package 'wheel' is not installed.
Using legacy 'setup.py install' for Mako, since package 'wheel' is not installed.
Using legacy 'setup.py install' for PrettyTable, since package 'wheel' is not installed.
Using legacy 'setup.py install' for pyperclip, since package 'wheel' is not installed.
  Attempting uninstall: pip
    Found existing installation: pip 20.2.2
    Uninstalling pip-20.2.2:
      Successfully uninstalled pip-20.2.2
    Running setup.py install for termcolor ... done
    Running setup.py install for sklearn ... done
    Running setup.py install for Mako ... done
    Running setup.py install for alembic ... done
    Running setup.py install for PrettyTable ... done
    Running setup.py 

In [2]:
!sudo pip3 install opencv-python imblearn

     |████████████████████████████████| 167 kB 2.7 MB/s eta 0:00:01


In [8]:
#!/usr/bin/env python3
import os
import logging
from pathlib import Path
import requests 
from glob import glob
from zipfile import ZipFile
import pickle
import pandas as pd

logging.basicConfig(level=logging.DEBUG)

#Import Pegasus API
from Pegasus.api import *
props = Properties()
props["pegasus.mode"] = "development"
props.write()

#Replica Catalog
rc = ReplicaCatalog()
in_files=[]

label_csv = './inputs/training_solutions_rev1.csv'

for file in glob("./inputs/*.jpg"):
    f = file.replace("./inputs/", '')
    in_files.append(File(f))
    rc.add_replica("local", File(f), Path("./inputs/").resolve() / f)
    
csv = label_csv.replace("./inputs/",'')
rc.add_replica("local", File(csv), Path("./inputs/").resolve() / csv)
rc.write()
#Transformation

data_label = Transformation( "Pegasus_Job1.py",
            site="local",
            pfn="/home/scitech/shared-data/Galaxy2/Pegasus_Job1.py",
            is_stageable=True
            )

tc = TransformationCatalog()\
    .add_transformations(data_label)\
    .write()

#Workflow
wf = Workflow("Galaxy", infer_dependencies=True)

train_dataset = []
val_dataset = []
test_dataset = []
dataset =[]

for i in range(93):
    train_dataset.append(File('train_'+ str(i)+'.jpg'))
for i in range(24):
    val_dataset.append(File('val_'+ str(i)+'.jpg'))
for i in range(30):
    test_dataset.append(File('test_'+ str(i)+'.jpg'))
    
dataset.extend(train_dataset)
dataset.extend(val_dataset)
dataset.extend(test_dataset)

Train_csv = File('Training_Data.csv')
Val_csv = File('Val_Data.csv')
Test_csv = File('Test_Data.csv')

job_data_label = Job(data_label)\
                    .add_inputs(*in_files, File(csv))\
                    .add_outputs(*dataset,Train_csv,Val_csv, Test_csv)

prrprocess 

wf.add_jobs(job_data_label)                                    

In [9]:
try:
     wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)


################
# pegasus-plan #
################
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2021.02.04 21:06:04.702 UTC:
2021.02.04 21:06:04.707 UTC:   -----------------------------------------------------------------------
2021.02.04 21:06:04.713 UTC:   File for submitting this DAG to HTCondor           : Galaxy-0.dag.condor.sub
2021.02.04 21:06:04.718 UTC:   Log of DAGMan debugging messages                 : Galaxy-0.dag.dagman.out
2021.02.04 21:06:04.723 UTC:   Log of HTCondor libra

[##################################################] 100.0% ..Success (Completed: 9, Queued: 0, Running: 0, Failed: 0)



####################
# pegasus-analyzer #
####################
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

Submit Directory   : /home/scitech/shared-data/Galaxy2/scitech/pegasus/Galaxy/run0001
Total jobs         :      9 (100.00%)
# jobs succeeded   :      9 (100.00%)
# jobs failed      :      0 (0.00%)
# jobs held        :      0 (0.00%)
# jobs unsubmitted :      0 (0.00%)



######################
# pegasus-statistics #
######################
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workflows.
#    